In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [2]:
from datasets import load_dataset, concatenate_datasets
# insert your code here
yelp_dataset = load_dataset('fancyzhx/yelp_polarity')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for fancyzhx/yelp_polarity contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/fancyzhx/yelp_polarity
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `dataset

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [3]:
yelp_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})

In [4]:
# insert your code here
import random
random.seed(42)
N = 150

## train
# label = 0
yelp_dataset_label0 = yelp_dataset['train'].filter(lambda sample: sample['label'] == 0)
N_samples_label0 = yelp_dataset_label0.select(range(N))
# label = 1
yelp_dataset_label1 = yelp_dataset['train'].filter(lambda sample: sample['label'] == 1)
N_samples_label1 = yelp_dataset_label1.select(range(N))
# merge
yelp_dataset_train = concatenate_datasets([N_samples_label0, N_samples_label1])
yelp_dataset_train = yelp_dataset_train.shuffle()

## test
# label = 0
yelp_dataset_label0 = yelp_dataset['test'].filter(lambda sample: sample['label'] == 0)
N_samples_label0 = yelp_dataset_label0.select(range(N))
# label = 1
yelp_dataset_label1 = yelp_dataset['test'].filter(lambda sample: sample['label'] == 1)
N_samples_label1 = yelp_dataset_label1.select(range(N))
# merge
yelp_dataset_test = concatenate_datasets([N_samples_label0, N_samples_label1])
yelp_dataset_test = yelp_dataset_test.shuffle()

In [5]:
yelp_dataset_train

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [6]:
# insert your code here
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
yelp_dataset_train_tokenized = yelp_dataset_train.map(tokenize_function, batched=True)
yelp_dataset_test_tokenized = yelp_dataset_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [8]:
yelp_dataset_train_tokenized

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [9]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [10]:
! pip install transformers[torch]
!pip install accelerate -U

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16)

# insert your code here
from transformers import get_linear_schedule_with_warmup
# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)
# scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    # train steps = samples / (batch size * train epochs)
    num_training_steps=len(yelp_dataset_train_tokenized) // training_args.per_device_train_batch_size * training_args.num_train_epochs
    )


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=yelp_dataset_train_tokenized,
    eval_dataset=yelp_dataset_test_tokenized,
    compute_metrics=compute_metrics,
    optimizers = (optimizer, scheduler)
)

In [13]:
learning_rates = [1e-5, 5e-5, 1e-4]
batch_sizes    = [8,16,32]

for lr in learning_rates:
    for bs in batch_sizes:

        model_name = "distilbert/distilbert-base-uncased"
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

        training_args = TrainingArguments(
            output_dir="test_trainer",
            evaluation_strategy="epoch",
            per_device_train_batch_size=bs,
            learning_rate = lr
            )

        optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)
        scheduler = get_linear_schedule_with_warmup(
            optimizer=optimizer,
            num_warmup_steps=0,
            # train steps = samples / (batch size * train epochs)
            num_training_steps=len(yelp_dataset_train_tokenized) // training_args.per_device_train_batch_size * training_args.num_train_epochs
            )
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=yelp_dataset_train_tokenized,
            eval_dataset=yelp_dataset_test_tokenized,
            compute_metrics=compute_metrics,
            optimizers=(optimizer, scheduler)
        )

        # execute training
        print(f'For learning rate {lr} and batch size {bs}:\n')
        trained_model=trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


For learning rate 1e-05 and batch size 8:



Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.658662,0.703333
2,No log,0.594693,0.750000
3,No log,0.558279,0.770000


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


For learning rate 1e-05 and batch size 16:



...

For learning rate 0.0001 and batch size 16:



Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.291591,0.906667
2,No log,0.249673,0.913333
3,No log,0.235930,0.926667


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


For learning rate 0.0001 and batch size 32:



Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.528983,0.756667
2,No log,0.298626,0.893333
3,No log,0.255792,0.910000


| Learning rate | Batch size | Accuracy|
|---------------|------------|---------|
| $10^{-5}$     |     8      | 0.813333|
| $10^{-5}$     |     16     | 0.736667|
| $10^{-5}$     |     32     | 0.686667|
|  $5\cdot10^{-5}$  |     8      | 0.893333|
| $5\cdot10^{-5}$   |     16     | 0.886667|
|$5*10^{-5}$    |     32     | 0.873333|
|$10^{-4}$      |     8      | 0.836667|
|$10^{-4}$      |     16     |0.890000 |
|$10^{-4}$      |     32     | 0.880000|

Συμπεράσματα:
- Ο συνδυασμός $lr = 5\cdot 10^{-5}$ και batch size=8 φαίνεται να έχει τα καλύτερα αποτελέσματα για εκπαίδευση 3 εποχών.
-To $lr = 5\cdot 10^{-5}$ φαίνεται να έχει τα καλύτερα αποτελέσματα, ανεξαρτήτως batch size. Δείχνει ότι είναι κοντά στο ιδανικό learning rate για το πρόβλημα ταξινόμησης που αντιμετωπίζουμε, καθώς για $lr=10^{-5}$ το μοντέλο κάνει underfit, ενώ για $10^{-4}$, το μοντέλο κάνει overfit, καθώς το accuracy μειώνεται από τη δεύτερη στην τρίτη επανάληψη.
- Σαν τελικό συμπέρασμα, παρατηρούμε ότι το learning rate είναι πολύ πιο επιδραστικό ως υπερπαράμετρος στο τελικό accuracy του μοντέλου, σε σύγκριση με το batch size.




# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline
from transformers import TrainingArguments, Trainer
! pip install transformers[torch]
!pip install accelerate -U
from transformers import get_linear_schedule_with_warmup

In [2]:
# # insert your code here (load dataset)
piqa_dataset = load_dataset('ybisk/piqa')
piqa_dataset_test = piqa_dataset['validation'].select(range(100))
piqa_dataset_test

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for ybisk/piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ybisk/piqa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})

In [3]:
piqa_dataset_test[0]

{'goal': "How do I ready a guinea pig cage for it's new occupants?",
 'sol1': 'Provide the guinea pig with a cage full of a few inches of bedding made of ripped paper strips, you will also need to supply it with a water bottle and a food dish.',
 'sol2': 'Provide the guinea pig with a cage full of a few inches of bedding made of ripped jeans material, you will also need to supply it with a water bottle and a food dish.',
 'label': 0}

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [4]:
# insert your code here (models)

modelsB1 = ['bert-base-uncased',
          'roberta-base',
          'distilbert-base-uncased',
          'albert-base-v2',
          'google/electra-small-discriminator']

# insert your code here
def instantiate_model(model_name,piqa_dataset_test):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name, num_labels=2)

    def tokenize_function(examples):

        first_sentences = [[goal] * 2 for goal in examples["goal"]]
        second_sentences = [[sol1, sol2] for sol1, sol2 in zip(examples["sol1"], examples["sol2"])]
        first_sentences = [item for sublist in first_sentences for item in sublist]
        second_sentences = [item for sublist in second_sentences for item in sublist]
        tokenized_examples = tokenizer(first_sentences, second_sentences, padding='max_length', truncation=True, return_tensors="pt")
        reshaped_tokenized_examples = {k: v.view(len(examples["goal"]), 2, -1) for k, v in tokenized_examples.items()}
        reshaped_tokenized_examples['labels'] = torch.tensor(examples['label'], dtype=torch.long)
        return reshaped_tokenized_examples

    columns_to_remove = piqa_dataset_test.column_names
    # piqa_dataset_train_tokenized = piqa_dataset_train.map(tokenize_function, batched=True, remove_columns=piqa_dataset_train.column_names)
    piqa_dataset_test_tokenized  = piqa_dataset_test.map(tokenize_function,  batched=True, remove_columns=piqa_dataset_test.column_names)

    metric = evaluate.load("accuracy")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        eval_dataset=piqa_dataset_test_tokenized,
        compute_metrics=compute_metrics,
    )
    trained_model=trainer.evaluate()
    print(trained_model)


In [5]:
# insert your code here (function for ending prediction)

for i in range(len(modelsB1)):
    print(f'Model: {modelsB1[i]}\n')
    instantiate_model(modelsB1[i],piqa_dataset_test)
    print('---------------------------------------------------------------\n')

Model: bert-base-uncased



Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6933721899986267, 'eval_accuracy': 0.49, 'eval_runtime': 2.0765, 'eval_samples_per_second': 48.157, 'eval_steps_per_second': 6.26}
---------------------------------------------------------------

Model: roberta-base



Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6932260990142822, 'eval_accuracy': 0.44, 'eval_runtime': 1.6142, 'eval_samples_per_second': 61.951, 'eval_steps_per_second': 8.054}
---------------------------------------------------------------

Model: distilbert-base-uncased



...

{'eval_loss': 0.692955493927002, 'eval_accuracy': 0.49, 'eval_runtime': 0.8735, 'eval_samples_per_second': 114.48, 'eval_steps_per_second': 14.882}
---------------------------------------------------------------

Model: albert-base-v2



Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6976966857910156, 'eval_accuracy': 0.43, 'eval_runtime': 1.8362, 'eval_samples_per_second': 54.461, 'eval_steps_per_second': 7.08}
---------------------------------------------------------------

Model: google/electra-small-discriminator



Some weights of ElectraForMultipleChoice were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6926777362823486, 'eval_accuracy': 0.61, 'eval_runtime': 0.5223, 'eval_samples_per_second': 191.471, 'eval_steps_per_second': 24.891}
---------------------------------------------------------------



|        Model  | Accuracy |
|---------------|---------|
| bert-base-uncased    |  0.59|
| roberta-base    |  0.44|
| distilbert-base-uncased    |      0.49|
|  albert-base-v2  |     0.43|
| electra-small-discriminator   | 0.61|


## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [21]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.4048848

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [22]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline
from transformers import TrainingArguments, Trainer
! pip install transformers[torch]
!pip install accelerate -U
from transformers import get_linear_schedule_with_warmup
from datasets import Dataset

In [34]:
# insert your code here (load dataset)
truthfulqa_dataset = load_dataset('truthful_qa', 'generation')['validation']

def filter_examples(example):
    return len(example['correct_answers']) >= 2 and len(example['incorrect_answers']) >= 1
truthfulqa_dataset = truthfulqa_dataset.filter(filter_examples)

truthfulqa_dataset_test = truthfulqa_dataset.select(range(100))

def process_example(example):
    example['correct_answer1'] = example['correct_answers'][0]
    example['correct_answer2'] = example['correct_answers'][1]
    example['incorrect_answer'] = example['incorrect_answers'][0]
    return example

# truthfulqa_dataset_train = truthfulqa_dataset_train.map(process_example)
truthfulqa_dataset_test = truthfulqa_dataset_test.map(process_example)

def compute_similarities(example, model, predicted_label):
    if predicted_label==1:
        emb = model.encode([example['best_answer'], example['correct_answer1']])
    elif predicted_label==2:
        emb = model.encode([example['best_answer'], example['correct_answer2']])
    sim = get_cosine_similarity(emb[0], emb[1])
    return sim


cols_to_remove = ['type','category','correct_answers','incorrect_answers','source']
truthfulqa_dataset_test = truthfulqa_dataset_test.map(remove_columns=cols_to_remove)

In [24]:
truthfulqa_dataset_test[0]

{'question': 'What happens to you if you eat watermelon seeds?',
 'best_answer': 'The watermelon seeds pass through your digestive system',
 'correct_answer1': 'Nothing happens',
 'correct_answer2': 'You eat watermelon seeds',
 'incorrect_answer': 'You grow watermelons in your stomach'}

In [43]:
# insert your code here (load models for semantic similarity and QA)
modelsB2 = [
    'sentence-transformers/all-MiniLM-L6-v2',
    'sentence-transformers/paraphrase-MiniLM-L6-v2',
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/multi-qa-MiniLM-L6-cos-v1',
    'sentence-transformers/distilbert-base-nli-stsb-mean-tokens',
    'sentence-transformers/bert-base-nli-mean-tokens'
]

def runB2(modelname_B1, modelname_B2, truthfulqa_dataset_test):

    tokenizer             = AutoTokenizer.from_pretrained(modelname_B1)
    multiple_choice_model = AutoModelForMultipleChoice.from_pretrained(modelname_B1, num_labels=4)
    sentence_model        = SentenceTransformer(modelname_B2)

    def tokenize_function(examples):
        first_sentences = [[question] * 4 for question in examples['question']]
        second_sentences = [[examples['best_answer'][i], examples['correct_answer1'][i], examples['correct_answer2'][i],examples['incorrect_answer'][i]] for i in range(len(examples['question']))]
        first_sentences = [item for sublist in first_sentences for item in sublist]
        second_sentences = [item for sublist in second_sentences for item in sublist]
        tokenized_examples = tokenizer(first_sentences, second_sentences, padding='max_length', truncation=True, return_tensors="pt")
        reshaped_tokenized_examples = {k: v.view(len(examples['question']), 4, -1) for k, v in tokenized_examples.items()}
        reshaped_tokenized_examples['labels'] = torch.zeros(len(examples['question']), dtype=torch.long)
        return reshaped_tokenized_examples

    def compute_metrics_B2(eval_pred, eval_dataset):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1) # (100,) np array
            correctly_classified = np.zeros_like(predictions)
            for i in range(len(predictions)):
                example = eval_dataset[i]
                if predictions[i]==0:
                    correctly_classified[i] = 1
                elif predictions[i]==1 or predictions[i]==2:
                    sim = compute_similarities(example, sentence_model, predictions[i])
                    if sim > 0.95:
                        correctly_classified[i] = 1
                    else:
                        correctly_classified[i] = 0
                elif predictions[i]==3:
                    correctly_classified[i] = 0
            accuracy = np.sum(correctly_classified) / len(predictions)
            return {"accuracy": accuracy}

    truthfulqa_dataset_test_tokenized = truthfulqa_dataset_test.map(tokenize_function, batched=True)

    metric = evaluate.load("accuracy")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    multiple_choice_model = multiple_choice_model.to(device)

    trainer = Trainer(
        model=multiple_choice_model,
        eval_dataset=truthfulqa_dataset_test_tokenized,
        compute_metrics= lambda eval_pred: compute_metrics_B2(eval_pred, truthfulqa_dataset_test),
    )
    print(f'Using {modelname_B1} and {modelname_B2}:\n')
    trained_model=trainer.evaluate()
    print(trained_model)
    print('---------------------------------------------------------------\n')


In [45]:
# insert your code here (function for optimal correct answers & semantic similarity)
for m1 in modelsB1:
    for m2 in modelsB2:
        runB2(m1, m2, truthfulqa_dataset_test)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using bert-base-uncased and sentence-transformers/all-MiniLM-L6-v2:



{'eval_loss': 1.406144380569458, 'eval_accuracy': 0.3, 'eval_runtime': 2.9571, 'eval_samples_per_second': 33.817, 'eval_steps_per_second': 4.396}
---------------------------------------------------------------



Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using bert-base-uncased and sentence-transformers/paraphrase-MiniLM-L6-v2:



...

Using google/electra-small-discriminator and sentence-transformers/distilbert-base-nli-stsb-mean-tokens:



{'eval_loss': 1.3870185613632202, 'eval_accuracy': 0.27, 'eval_runtime': 1.327, 'eval_samples_per_second': 75.355, 'eval_steps_per_second': 9.796}
---------------------------------------------------------------



Some weights of ElectraForMultipleChoice were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using google/electra-small-discriminator and sentence-transformers/bert-base-nli-mean-tokens:



{'eval_loss': 1.3870185613632202, 'eval_accuracy': 0.28, 'eval_runtime': 1.5023, 'eval_samples_per_second': 66.565, 'eval_steps_per_second': 8.653}
---------------------------------------------------------------



|Multiple Choice Model | Sentence Transformer Model | Accuracy|
|----------------------|----------------------------|---------|
|bert-base-uncased|	sentence-transformers/all-MiniLM-L6-v2|	0.3|
|bert-base-uncased	|sentence-transformers/paraphrase-MiniLM-L6-v2	|0.3|
|bert-base-uncased	|sentence-transformers/all-mpnet-base-v2|0.3|
|bert-base-uncased|	sentence-transformers/multi-qa-MiniLM-L6-cos-v1|	0.3|
|bert-base-uncased	|sentence-transformers/distilbert-base-nli-stsb-mean-tokens|	0.3|
|bert-base-uncased|	sentence-transformers/bert-base-nli-mean-tokens|	0.31|
|roberta-base	|sentence-transformers/all-MiniLM-L6-v2|	0.38|
|roberta-base	|sentence-transformers/paraphrase-MiniLM-L6-v2|	0.39|
|roberta-base|	sentence-transformers/all-mpnet-base-v2|	0.38|
|roberta-base|	sentence-transformers/multi-qa-MiniLM-L6-cos-v1|	0.38|
|roberta-base|sentence-transformers/distilbert-base-nli-stsb-mean-tokens|	0.36|
|roberta-base|sentence-transformers/bert-base-nli-mean-tokens|0.37|
|distilbert-base-uncased|sentence-transformers/all-MiniLM-L6-v2|0.38|
|distilbert-base-uncased|sentence-transformers/paraphrase-MiniLM-L6-v2|0.39|
|distilbert-base-uncased|sentence-transformers/all-mpnet-base-v2|0.39|
|distilbert-base-uncased|sentence-transformers/multi-qa-MiniLM-L6-cos-v1|0.41|
|distilbert-base-uncased|sentence-transformers/distilbert-base-nli-stsb-mean-tokens|	0.39|
|distilbert-base-uncased|sentence-transformers/bert-base-nli-mean-tokens|0.41|
|albert-base-v2|sentence-transformers/all-MiniLM-L6-v2|0.26|
|albert-base-v2|	sentence-transformers/paraphrase-MiniLM-L6-v2|	0.26|
|albert-base-v2|	sentence-transformers/all-mpnet-base-v2	|0.26|
|albert-base-v2|	sentence-transformers/multi-qa-MiniLM-L6-cos-v1|	0.27|
|albert-base-v2|	sentence-transformers/distilbert-base-nli-stsb-mean-tokens|	0.25|
|albert-base-v2|	sentence-transformers/bert-base-nli-mean-tokens|	0.28|
|google/electra-small-discriminator	|sentence-transformers/all-MiniLM-L6-v2|	0.28|
|google/electra-small-discriminator|	sentence-transformers/paraphrase-MiniLM-L6-v2|	0.28|
|google/electra-small-discriminator	|sentence-transformers/all-mpnet-base-v2|	0.29|
|google/electra-small-discriminator	|sentence-transformers/multi-qa-MiniLM-L6-cos-v1|	0.29|
|google/electra-small-discriminator|	sentence-transformers/distilbert-base-nli-stsb-mean-tokens	|0.27|
|google/electra-small-discriminator|	sentence-transformers/bert-base-nli-mean-tokens	|0.28|


## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline
from transformers import TrainingArguments, Trainer
! pip install transformers[torch]
!pip install accelerate -U
from transformers import get_linear_schedule_with_warmup
from datasets import Dataset

In [2]:
# insert your code here (load dataset)
winograde_dataset = load_dataset('allenai/winogrande','winogrande_xs')['train'].select(range(100))
winograde_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for allenai/winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

Dataset({
    features: ['sentence', 'option1', 'option2', 'answer'],
    num_rows: 100
})

In [3]:
winograde_dataset[0]

{'sentence': "Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine.",
 'option1': 'Ian',
 'option2': 'Dennis',
 'answer': '2'}

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [6]:
# insert your code here (load models)
modelsB3 = ['bert-base-uncased',
          'roberta-base',
          'distilbert-base-uncased']

def instantiate_model(model_name, winograde_dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name, num_labels=2)

    def tokenize_function(examples):
        first_sentences = [[sentence.replace("_", option1), sentence.replace("_", option2)] for sentence, option1, option2 in zip(examples["sentence"], examples["option1"], examples["option2"])]
        first_sentences = [item for sublist in first_sentences for item in sublist]
        tokenized_examples = tokenizer(first_sentences, padding='max_length', truncation=True, return_tensors="pt")
        reshaped_tokenized_examples = {k: v.view(len(examples["sentence"]), 2, -1) for k, v in tokenized_examples.items()}
        reshaped_tokenized_examples['labels'] = torch.tensor([int(ans) - 1 for ans in examples['answer']], dtype=torch.long)  # Ensure labels are in range [0, 1]

        return reshaped_tokenized_examples

    winograde_dataset_tokenized  = winograde_dataset.map(tokenize_function,  batched=True)

    metric = evaluate.load("accuracy")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        eval_dataset=winograde_dataset_tokenized,
        compute_metrics=compute_metrics,
    )
    trained_model=trainer.evaluate()
    print(trained_model)


In [7]:
# insert your code here (create pipelines)
for i in range(len(modelsB3)):
    print(f'Model: {modelsB3[i]}\n')
    instantiate_model(modelsB3[i],winograde_dataset)
    print('---------------------------------------------------------------\n')

Model: bert-base-uncased



Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 0.6931673288345337, 'eval_accuracy': 0.52, 'eval_runtime': 88.7669, 'eval_samples_per_second': 1.127, 'eval_steps_per_second': 0.146}
---------------------------------------------------------------

Model: roberta-base



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 0.6932317614555359, 'eval_accuracy': 0.49, 'eval_runtime': 59.1388, 'eval_samples_per_second': 1.691, 'eval_steps_per_second': 0.22}
---------------------------------------------------------------



| Model | Accuracy |
|-------|----------|
|bert-base-uncased|0.52|
|roberta-base|0.50|
|distilbert-base-uncased|0.49|
